# Validação das regras de detecção da ameaça APT29 cenário 1

## Iniciando a sessão Spark e carregando o Dataset

In [1]:
#Iniciando a sessão Spark
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
spark.conf.set("spark.sql.caseSensitive", "true")

#Carregando o Dataset
df = spark.read.json('apt29_evals_day1_manual_2020-05-01225525.json')
df.createTempView('apt29')

## DTR-0017-3103-4080
TTP de referência: TTP-0017-3068-5003

In [13]:
df = spark.sql(
'''
SELECT `@timestamp`,UtcTime,Message
FROM apt29
WHERE Channel = "Microsoft-Windows-Sysmon/Operational"
    AND EventID = 1
    AND LOWER(ParentImage) LIKE "%explorer.exe"
    AND LOWER(Image) LIKE '%cod.%'
    AND LOWER(Image) LIKE '%.scr'
'''
)
df.show(100,truncate = False, vertical = True)

-RECORD 0------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 @timestamp | 2020-05-02T02:55:57.730Z                                                                                  

## DTR-0017-6092-4557
TTP de referência: TTP-0017-6044-2942

In [19]:
df = spark.sql(
'''
SELECT `@timestamp`,UtcTime,EventTime,Message
FROM apt29
WHERE Channel = "Microsoft-Windows-Sysmon/Operational"
    AND EventID = 3
    AND LOWER(DestinationPort) LIKE '1234'
    AND LOWER(Image) LIKE '%cod.%'
    AND LOWER(Image) LIKE '%.scr'
'''
)
df.show(100,truncate = False, vertical = True)

-RECORD 0--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 @timestamp | 2020-05-02T02:56:02.783Z                                                                                                                                                                                                                                                                                                                                                                                                                                                               
 UtcTime    | 2020-05-02 02:

## DTR-0017-6092-7552
TTP de referência: TTP-0017-6044-6572

In [18]:
df = spark.sql(
'''
SELECT `@timestamp`,UtcTime,EventTime,Message
FROM apt29 a
INNER JOIN (
    SELECT ProcessGuid
    FROM apt29
    WHERE Channel = "Microsoft-Windows-Sysmon/Operational"
        AND EventID = 1
        AND LOWER(ParentImage) LIKE '%cod.%'
        AND LOWER(ParentImage) LIKE '%.scr'
        AND LOWER(Image) LIKE '%cmd.exe'
) b
ON a.ParentProcessGuid = b.ProcessGuid
WHERE Channel = "Microsoft-Windows-Sysmon/Operational"
    AND EventID = 1
    AND LOWER(Image) LIKE '%powershell.exe'
'''
)
df.show(100,truncate = False, vertical = True)

-RECORD 0-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 @tim

## DTR-0017-6099-4535
TTP de referência: TTP-0017-6044-6947

In [17]:
df = spark.sql(
'''
SELECT b.`@timestamp`,b.EventTime,b.ScriptBlockText
FROM apt29 a
INNER JOIN (
  SELECT d.ParentProcessGuid, d.ProcessId, c.ScriptBlockText, c.EventTime, c.`@timestamp`
  FROM apt29 c
  INNER JOIN (
      SELECT ParentProcessGuid, ProcessGuid, ProcessId
      FROM apt29
      WHERE Channel = "Microsoft-Windows-Sysmon/Operational"
          AND EventID = 1
      ) d
  ON c.ExecutionProcessID = d.ProcessId
  WHERE c.Channel = "Microsoft-Windows-PowerShell/Operational"
          AND c.EventID = 4104
          AND LOWER(c.ScriptBlockText) LIKE "%childitem%"
) b
ON a.ProcessGuid = b.ParentProcessGuid
WHERE a.Channel = "Microsoft-Windows-Sysmon/Operational"
          AND a.EventID = 1
          AND LOWER(a.ParentImage) LIKE '%cod.%'
          AND LOWER(a.ParentImage) LIKE '%.scr'
'''
)
df.show(100,truncate = False, vertical = True)

-RECORD 0----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 @timestamp      | 2020-05-02T02:56:18.140Z                                                                                                                                                                                                                                                                                                                                                                                                                                    
 EventTime       | 2020-05-01 22:56:17                                  

## DTR-0017-6099-9496
TTP de referência: TTP-0017-6044-7344

In [9]:
df = spark.sql(
'''
SELECT b.`@timestamp`,b.EventTime,b.ScriptBlockText
FROM apt29 a
INNER JOIN (
  SELECT d.ParentProcessGuid, d.ProcessId, c.ScriptBlockText, c.EventTime, c.`@timestamp`
  FROM apt29 c
  INNER JOIN (
      SELECT ParentProcessGuid, ProcessGuid, ProcessId
      FROM apt29
      WHERE Channel = "Microsoft-Windows-Sysmon/Operational"
          AND EventID = 1
      ) d
  ON c.ExecutionProcessID = d.ProcessId
  WHERE c.Channel = "Microsoft-Windows-PowerShell/Operational"
          AND c.EventID = 4104
          AND LOWER(c.ScriptBlockText) LIKE "%compress-archive%"
) b
ON a.ProcessGuid = b.ParentProcessGuid
WHERE a.Channel = "Microsoft-Windows-Sysmon/Operational"
          AND a.EventID = 1
          AND LOWER(a.ParentImage) LIKE '%cod.%'
          AND LOWER(a.ParentImage) LIKE '%.scr'
'''
)
df.show(100,truncate = False, vertical = True)

-RECORD 0----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 @timestamp      | 2020-05-02T02:56:18.140Z                                                                                                                                                                                                                                                                                                                                                                                                                                    
 EventTime       | 2020-05-01 22:56:17                                  

## DTR-0017-6124-7922
TTP de referência: TTP-0017-6044-7344

In [42]:
df = spark.sql(
'''
SELECT `@timestamp`,UtcTime,EventTime,Message
FROM apt29 d
INNER JOIN (
    SELECT b.ProcessGuid
    FROM apt29 b
    INNER JOIN (
        SELECT ProcessGuid
        FROM apt29
        WHERE Channel = "Microsoft-Windows-Sysmon/Operational"
            AND EventID = 1
            AND LOWER(ParentImage) LIKE '%cod.%'
            AND LOWER(ParentImage) LIKE '%.scr'
    ) a
    ON b.ParentProcessGuid = a.ProcessGuid
    WHERE b.Channel = "Microsoft-Windows-Sysmon/Operational"
        AND b.EventID = 1
) c
ON d.ProcessGuid = c.ProcessGuid
WHERE d.Channel = "Microsoft-Windows-Sysmon/Operational"
  AND d.EventID = 11
  AND LOWER(d.TargetFilename) LIKE '%.zip'
'''
)
df.show(100,truncate = False, vertical = True)

-RECORD 0----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 @timestamp | 2020-05-02T02:56:20.426Z                                                                                                                                                                                                                                                                                   
 UtcTime    | 2020-05-02 02:56:18.032                                                                                                                                                                                                                                                                                    
 EventTime  | 2020-05-01 22:56:18                         

## DTR-0017-6100-1386
TTP de referência: TTP-0017-6044-7700

In [21]:
df = spark.sql(
'''
SELECT b.`@timestamp`,b.UtcTime,b.EventTime,b.Message
FROM apt29 a
INNER JOIN (
    SELECT ProcessGuid, Message, UtcTime, EventTime, `@timestamp`
    FROM apt29
    WHERE Channel = "Microsoft-Windows-Sysmon/Operational"
        AND EventID = 11
        AND LOWER(TargetFilename) LIKE '%.png'
) b
ON a.ProcessGuid = b.ProcessGuid
WHERE a.Channel = "Microsoft-Windows-Sysmon/Operational"
    AND a.EventID = 1
    AND LOWER(a.Image) LIKE '%cod.%'
    AND LOWER(a.Image) LIKE '%.scr'
'''
)
df.show(100,truncate = False, vertical = True)

-RECORD 0----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 @timestamp | 2020-05-02T02:57:01.833Z                                                                                                                                                                                                                                                           
 UtcTime    | 2020-05-02 02:57:00.933                                                                                                                                                                                                                                                            
 EventTime  | 2020-05-01 22:57:00                                                                                                 

## DTR-0017-6100-4576
TTP de referência: TTP-0017-6045-0689

In [22]:
df = spark.sql(
'''
SELECT `@timestamp`,UtcTime,EventTime,Message
FROM apt29
WHERE Channel = "Microsoft-Windows-Sysmon/Operational"
    AND EventID = 13
    AND LOWER(TargetObject) RLIKE '.*\\\\\\\\folder\\\\\\\\shell\\\\\\\\open\\\\\\\\command\\\\\\\delegateexecute.*'
'''
)
df.show(100,truncate = False, vertical = True)

-RECORD 0---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 @timestamp | 2020-05-02T02:58:32.662Z                                                                                                                                                                                                                                                                                                                                              
 UtcTime    | 2020-05-02 02:58:30.649                                                                                                                                                                                                                         

## DTR-0017-6100-6059
TTP de referência: TTP-0017-6045-0689

In [23]:
df = spark.sql(
'''
SELECT `@timestamp`,UtcTime,EventTime,Message
FROM apt29
WHERE Channel = "Microsoft-Windows-Sysmon/Operational"
    AND EventID = 1
    AND LOWER(Image) LIKE "%sdclt.exe"
    AND IntegrityLevel = "High"
'''
)
df.show(100,truncate = False, vertical = True)

-RECORD 0-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 @timestamp

## DTR-0017-6104-7163
TTP de referência: TTP-0017-6045-2305

In [24]:
df = spark.sql(
'''
SELECT `@timestamp`,UtcTime,EventTime,Message
FROM apt29 d
INNER JOIN (
    SELECT b.ProcessGuid
    FROM apt29 b
    INNER JOIN (
        SELECT ProcessGuid
        FROM apt29
        WHERE Channel = "Microsoft-Windows-Sysmon/Operational"
            AND EventID = 1
            AND LOWER(ParentImage) LIKE '%cod.%'
            AND LOWER(ParentImage) LIKE '%.scr'
    ) a
    ON b.ParentProcessGuid = a.ProcessGuid
    WHERE b.Channel = "Microsoft-Windows-Sysmon/Operational"
        AND b.EventID = 1
) c
ON d.ProcessGuid = c.ProcessGuid
WHERE d.Channel = "Microsoft-Windows-Sysmon/Operational"
  AND d.EventID = 12
  AND LOWER(d.TargetObject) RLIKE '.*\\\\\\\\folder\\\\\\\\shell\\\\\\\\open\\\\\\\\command.*'
  AND d.Message RLIKE '.*EventType: DeleteKey.*'
'''
)
df.show(100,truncate = False, vertical = True)

-RECORD 0--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 @timestamp | 2020-05-02T02:59:16.772Z                                                                                                                                                                                                                                                                                                                           
 UtcTime    | 2020-05-02 02:59:15.911                                                                                                                                                                                                                                                               

## DTR-0017-6104-8503
TTP de referência: TTP-0017-6045-2548

In [41]:
df = spark.sql(
'''
SELECT `@timestamp`,UtcTime,EventTime,Message
FROM apt29 d
INNER JOIN (
  SELECT a.ProcessGuid, a.ParentProcessGuid
  FROM apt29 a
  INNER JOIN (
    SELECT ProcessGuid
    FROM apt29
    WHERE Channel = "Microsoft-Windows-Sysmon/Operational"
        AND EventID = 1
        AND LOWER(Image) LIKE "%control.exe"
        AND LOWER(ParentImage) LIKE "%sdclt.exe"
  ) b
  ON a.ParentProcessGuid = b.ProcessGuid
  WHERE a.Channel = "Microsoft-Windows-Sysmon/Operational"
    AND a.EventID = 1
    AND a.IntegrityLevel = "High"
) c
ON d.ProcessGuid = c.ProcessGuid
WHERE d.Channel = "Microsoft-Windows-Sysmon/Operational"
  AND d.EventID = 11
  AND LOWER(d.TargetFilename) LIKE '%.zip'
'''
)
df.show(100,truncate = False, vertical = True)

-RECORD 0----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 @timestamp | 2020-05-02T02:59:44.582Z                                                                                                                                                                                                                                                                                         
 UtcTime    | 2020-05-02 02:59:42.544                                                                                                                                                                                                                                                                                          
 EventTime  | 2020-05-01 22:59:42       

## DTR-0017-6104-9721
TTP de referência: TTP-0017-6045-8979

In [27]:
df = spark.sql(
'''
SELECT `@timestamp`,EventTime,Payload
FROM apt29 f
INNER JOIN (
    SELECT d.ProcessId
    FROM apt29 d
    INNER JOIN (
      SELECT a.ProcessGuid, a.ParentProcessGuid
      FROM apt29 a
      INNER JOIN (
        SELECT ProcessGuid
        FROM apt29
        WHERE Channel = "Microsoft-Windows-Sysmon/Operational"
            AND EventID = 1
            AND LOWER(Image) LIKE "%control.exe"
            AND LOWER(ParentImage) LIKE "%sdclt.exe"
      ) b
      ON a.ParentProcessGuid = b.ProcessGuid
      WHERE a.Channel = "Microsoft-Windows-Sysmon/Operational"
        AND a.EventID = 1
        AND a.IntegrityLevel = "High"
    ) c
    ON d.ParentProcessGuid= c.ProcessGuid
    WHERE d.Channel = "Microsoft-Windows-Sysmon/Operational"
      AND d.EventID = 1
      AND d.Image LIKE '%powershell.exe'
) e
ON f.ExecutionProcessID = e.ProcessId
WHERE f.Channel = "Microsoft-Windows-PowerShell/Operational"
    AND f.EventID = 4103
    AND LOWER(f.Payload) LIKE "%expand-archive%"
'''
)
df.show(100,truncate = False, vertical = True)

-RECORD 0--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 @timestamp | 2020-05-02T03:00:37.963Z                                                                                                                                                                                                                                                                                                                                                  

## DTR-0017-6105-0296
TTP de referência: TTP-0017-6045-9266

In [43]:
df = spark.sql(
'''
SELECT `@timestamp`,EventTime,ScriptBlockText
FROM apt29 f
INNER JOIN (
    SELECT d.ProcessId
    FROM apt29 d
    INNER JOIN (
      SELECT a.ProcessGuid, a.ParentProcessGuid
      FROM apt29 a
      INNER JOIN (
        SELECT ProcessGuid
        FROM apt29
        WHERE Channel = "Microsoft-Windows-Sysmon/Operational"
            AND EventID = 1
            AND LOWER(Image) LIKE "%control.exe"
            AND LOWER(ParentImage) LIKE "%sdclt.exe"
      ) b
      ON a.ParentProcessGuid = b.ProcessGuid
      WHERE a.Channel = "Microsoft-Windows-Sysmon/Operational"
        AND a.EventID = 1
        AND a.IntegrityLevel = "High"
    ) c
    ON d.ParentProcessGuid= c.ProcessGuid
    WHERE d.Channel = "Microsoft-Windows-Sysmon/Operational"
      AND d.EventID = 1
      AND d.Image LIKE '%powershell.exe'
) e
ON f.ExecutionProcessID = e.ProcessId
WHERE f.Channel = "Microsoft-Windows-PowerShell/Operational"
    AND f.EventID = 4104
    AND LOWER(f.ScriptBlockText) LIKE "%get-process%"
'''
)
df.show(100,truncate = False, vertical = True)

-RECORD 0-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

## DTR-0017-6105-0810
TTP de referência: TTP-0017-6046-1439

In [44]:
df = spark.sql(
'''
SELECT `@timestamp`,UtcTime,EventTime,Message, g.CommandLine
FROM apt29 h
INNER JOIN (
  SELECT f.ProcessGuid, f.CommandLine
  FROM apt29 f
  INNER JOIN (
    SELECT d.ProcessId, d.ProcessGuid
    FROM apt29 d
    INNER JOIN (
      SELECT a.ProcessGuid, a.ParentProcessGuid
      FROM apt29 a
      INNER JOIN (
        SELECT ProcessGuid
        FROM apt29
        WHERE Channel = "Microsoft-Windows-Sysmon/Operational"
            AND EventID = 1
            AND LOWER(Image) LIKE "%control.exe"
            AND LOWER(ParentImage) LIKE "%sdclt.exe"
      ) b
      ON a.ParentProcessGuid = b.ProcessGuid
      WHERE a.Channel = "Microsoft-Windows-Sysmon/Operational"
        AND a.EventID = 1
        AND a.IntegrityLevel = "High"
    ) c
    ON d.ParentProcessGuid = c.ProcessGuid
    WHERE d.Channel = "Microsoft-Windows-Sysmon/Operational"
      AND d.EventID = 1
      AND d.Image LIKE '%powershell.exe'
  ) e
  ON f.ParentProcessGuid = e.ProcessGuid
  WHERE f.Channel = "Microsoft-Windows-Sysmon/Operational"
    AND f.EventID = 1
    AND LOWER(f.Image) LIKE '%sdelete%'
    AND (
        LOWER(f.CommandLine) LIKE '%draft.zip%'
        OR LOWER(f.CommandLine) LIKE '%sysinternalssuite.zip%'
        OR (
            LOWER(f.CommandLine) LIKE '%cod.%'
            AND LOWER(f.CommandLine) LIKE '%.scr'
        )
    )
) g
ON h.ProcessGuid = g.ProcessGuid
WHERE h.Channel = "Microsoft-Windows-Sysmon/Operational"
  AND h.EventID = 23
'''
)
df.show(100,truncate = False, vertical = True)

-RECORD 0--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 @timestamp  | 2020-05-02T03:03:39.433Z                                                                                                                                                                                                                                                                                                                                                                                                                                           

## DTR-0017-6105-2497
TTP de referência: TTP-0017-6046-1657

In [45]:
df = spark.sql(
'''
SELECT `@timestamp`,EventTime,ScriptBlockText
FROM apt29 f
INNER JOIN (
  SELECT d.ProcessId
  FROM apt29 d
  INNER JOIN (
    SELECT a.ProcessGuid, a.ParentProcessGuid
    FROM apt29 a
    INNER JOIN (
      SELECT ProcessGuid
      FROM apt29
      WHERE Channel = "Microsoft-Windows-Sysmon/Operational"
          AND EventID = 1
          AND LOWER(Image) LIKE "%control.exe"
          AND LOWER(ParentImage) LIKE "%sdclt.exe"
    ) b
    ON a.ParentProcessGuid = b.ProcessGuid
    WHERE a.Channel = "Microsoft-Windows-Sysmon/Operational"
      AND a.EventID = 1
      AND a.IntegrityLevel = "High"
  ) c
  ON d.ParentProcessGuid= c.ProcessGuid
  WHERE d.Channel = "Microsoft-Windows-Sysmon/Operational"
    AND d.EventID = 1
    AND d.Image LIKE '%powershell.exe'
) e
ON f.ExecutionProcessID = e.ProcessId
WHERE f.Channel = "Microsoft-Windows-PowerShell/Operational"
  AND f.EventID = 4104
  AND LOWER(f.ScriptBlockText) LIKE "%$env:temp%"
'''
)
df.show(100,truncate = False, vertical = True)

-RECORD 0-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

## DTR-0017-6105-3592
TTP de referência: TTP-0017-6046-2409

In [46]:
df = spark.sql(
'''
SELECT `@timestamp`,EventTime,ScriptBlockText
FROM apt29 f
INNER JOIN (
  SELECT d.ProcessId
  FROM apt29 d
  INNER JOIN (
    SELECT a.ProcessGuid, a.ParentProcessGuid
    FROM apt29 a
    INNER JOIN (
      SELECT ProcessGuid
      FROM apt29
      WHERE Channel = "Microsoft-Windows-Sysmon/Operational"
          AND EventID = 1
          AND LOWER(Image) LIKE "%control.exe"
          AND LOWER(ParentImage) LIKE "%sdclt.exe"
    ) b
    ON a.ParentProcessGuid = b.ProcessGuid
    WHERE a.Channel = "Microsoft-Windows-Sysmon/Operational"
      AND a.EventID = 1
      AND a.IntegrityLevel = "High"
  ) c
  ON d.ParentProcessGuid= c.ProcessGuid
  WHERE d.Channel = "Microsoft-Windows-Sysmon/Operational"
    AND d.EventID = 1
    AND d.Image LIKE '%powershell.exe'
) e
ON f.ExecutionProcessID = e.ProcessId
WHERE f.Channel = "Microsoft-Windows-PowerShell/Operational"
  AND f.EventID = 4104
  AND LOWER(f.ScriptBlockText) LIKE "%$env:username%"
'''
)
df.show(100,truncate = False, vertical = True)

-RECORD 0-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

## DTR-0017-6105-5552
TTP de referência: TTP-0017-6046-2437

In [47]:
df = spark.sql(
'''
SELECT `@timestamp`,EventTime,ScriptBlockText
FROM apt29 f
INNER JOIN (
  SELECT d.ProcessId
  FROM apt29 d
  INNER JOIN (
    SELECT a.ProcessGuid, a.ParentProcessGuid
    FROM apt29 a
    INNER JOIN (
      SELECT ProcessGuid
      FROM apt29
      WHERE Channel = "Microsoft-Windows-Sysmon/Operational"
          AND EventID = 1
          AND LOWER(Image) LIKE "%control.exe"
          AND LOWER(ParentImage) LIKE "%sdclt.exe"
    ) b
    ON a.ParentProcessGuid = b.ProcessGuid
    WHERE a.Channel = "Microsoft-Windows-Sysmon/Operational"
      AND a.EventID = 1
      AND a.IntegrityLevel = "High"
  ) c
  ON d.ParentProcessGuid= c.ProcessGuid
  WHERE d.Channel = "Microsoft-Windows-Sysmon/Operational"
    AND d.EventID = 1
    AND d.Image LIKE '%powershell.exe'
) e
ON f.ExecutionProcessID = e.ProcessId
WHERE f.Channel = "Microsoft-Windows-PowerShell/Operational"
  AND f.EventID = 4104
  AND (
    LOWER(f.ScriptBlockText) LIKE "%$env:computername%"
    OR LOWER(f.ScriptBlockText) LIKE "%gwmi win32_operatingsystem%"
    )
'''
)
df.show(100,truncate = False, vertical = True)

-RECORD 0-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

## DTR-0017-6105-6366
TTP de referência: TTP-0017-6046-2449

In [48]:
df = spark.sql(
'''
SELECT `@timestamp`,EventTime,ScriptBlockText
FROM apt29 f
INNER JOIN (
  SELECT d.ProcessId
  FROM apt29 d
  INNER JOIN (
    SELECT a.ProcessGuid, a.ParentProcessGuid
    FROM apt29 a
    INNER JOIN (
      SELECT ProcessGuid
      FROM apt29
      WHERE Channel = "Microsoft-Windows-Sysmon/Operational"
          AND EventID = 1
          AND LOWER(Image) LIKE "%control.exe"
          AND LOWER(ParentImage) LIKE "%sdclt.exe"
    ) b
    ON a.ParentProcessGuid = b.ProcessGuid
    WHERE a.Channel = "Microsoft-Windows-Sysmon/Operational"
      AND a.EventID = 1
      AND a.IntegrityLevel = "High"
  ) c
  ON d.ParentProcessGuid= c.ProcessGuid
  WHERE d.Channel = "Microsoft-Windows-Sysmon/Operational"
    AND d.EventID = 1
    AND d.Image LIKE '%powershell.exe'
) e
ON f.ExecutionProcessID = e.ProcessId
WHERE f.Channel = "Microsoft-Windows-PowerShell/Operational"
  AND f.EventID = 4104
  AND LOWER(f.ScriptBlockText) LIKE "%$env:userdomain%"
'''
)
df.show(100,truncate = False, vertical = True)

-RECORD 0-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

## DTR-0017-6105-6754
TTP de referência: TTP-0017-6046-2450

In [49]:
df = spark.sql(
'''
SELECT `@timestamp`,EventTime,ScriptBlockText
FROM apt29 f
INNER JOIN (
  SELECT d.ProcessId
  FROM apt29 d
  INNER JOIN (
    SELECT a.ProcessGuid, a.ParentProcessGuid
    FROM apt29 a
    INNER JOIN (
      SELECT ProcessGuid
      FROM apt29
      WHERE Channel = "Microsoft-Windows-Sysmon/Operational"
          AND EventID = 1
          AND LOWER(Image) LIKE "%control.exe"
          AND LOWER(ParentImage) LIKE "%sdclt.exe"
    ) b
    ON a.ParentProcessGuid = b.ProcessGuid
    WHERE a.Channel = "Microsoft-Windows-Sysmon/Operational"
      AND a.EventID = 1
      AND a.IntegrityLevel = "High"
  ) c
  ON d.ParentProcessGuid= c.ProcessGuid
  WHERE d.Channel = "Microsoft-Windows-Sysmon/Operational"
    AND d.EventID = 1
    AND d.Image LIKE '%powershell.exe'
) e
ON f.ExecutionProcessID = e.ProcessId
WHERE f.Channel = "Microsoft-Windows-PowerShell/Operational"
  AND f.EventID = 4104
  AND LOWER(f.ScriptBlockText) LIKE "%$pid%"
'''
)
df.show(100,truncate = False, vertical = True)

-RECORD 0-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

## DTR-0017-6105-7294
TTP de referência: TTP-0017-6046-2453

In [50]:
df = spark.sql(
'''
SELECT `@timestamp`,EventTime,ScriptBlockText
FROM apt29 f
INNER JOIN (
  SELECT d.ProcessId
  FROM apt29 d
  INNER JOIN (
    SELECT a.ProcessGuid, a.ParentProcessGuid
    FROM apt29 a
    INNER JOIN (
      SELECT ProcessGuid
      FROM apt29
      WHERE Channel = "Microsoft-Windows-Sysmon/Operational"
          AND EventID = 1
          AND LOWER(Image) LIKE "%control.exe"
          AND LOWER(ParentImage) LIKE "%sdclt.exe"
    ) b
    ON a.ParentProcessGuid = b.ProcessGuid
    WHERE a.Channel = "Microsoft-Windows-Sysmon/Operational"
      AND a.EventID = 1
      AND a.IntegrityLevel = "High"
  ) c
  ON d.ParentProcessGuid= c.ProcessGuid
  WHERE d.Channel = "Microsoft-Windows-Sysmon/Operational"
    AND d.EventID = 1
    AND d.Image LIKE '%powershell.exe'
) e
ON f.ExecutionProcessID = e.ProcessId
WHERE f.Channel = "Microsoft-Windows-PowerShell/Operational"
  AND f.EventID = 4104
  AND (
    LOWER(f.ScriptBlockText) LIKE "%-class antivirusproduct%"
    OR LOWER(f.ScriptBlockText) LIKE "%-class firewallproduct%"
    )
'''
)
df.show(100,truncate = False, vertical = True)

-RECORD 0-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

## DTR-0017-6106-2801
TTP de referência: TTP-0017-6046-2456

In [51]:
df = spark.sql(
'''
SELECT `@timestamp`,EventTime,ScriptBlockText
FROM apt29 f
INNER JOIN (
  SELECT d.ProcessId
  FROM apt29 d
  INNER JOIN (
    SELECT a.ProcessGuid, a.ParentProcessGuid
    FROM apt29 a
    INNER JOIN (
      SELECT ProcessGuid
      FROM apt29
      WHERE Channel = "Microsoft-Windows-Sysmon/Operational"
          AND EventID = 1
          AND LOWER(Image) LIKE "%control.exe"
          AND LOWER(ParentImage) LIKE "%sdclt.exe"
    ) b
    ON a.ParentProcessGuid = b.ProcessGuid
    WHERE a.Channel = "Microsoft-Windows-Sysmon/Operational"
      AND a.EventID = 1
      AND a.IntegrityLevel = "High"
  ) c
  ON d.ParentProcessGuid= c.ProcessGuid
  WHERE d.Channel = "Microsoft-Windows-Sysmon/Operational"
    AND d.EventID = 1
    AND d.Image LIKE '%powershell.exe'
) e
ON f.ExecutionProcessID = e.ProcessId
WHERE f.Channel = "Microsoft-Windows-PowerShell/Operational"
  AND f.EventID = 4104
  AND (
    LOWER(f.ScriptBlockText) LIKE "%netusergetgroups%"
    OR LOWER(f.ScriptBlockText) LIKE "%netusergetlocalgroups%"
    )
'''
)
df.show(100,truncate = False, vertical = True)

-RECORD 0-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

## DTR-0017-6106-4244
TTP de referência: TTP-0017-6046-4431

In [52]:
df = spark.sql(
'''
SELECT `@timestamp`,EventTime,Payload
FROM apt29 f
INNER JOIN (
  SELECT d.ProcessId, d.ParentProcessId
  FROM apt29 d
  INNER JOIN (
    SELECT a.ProcessGuid, a.ParentProcessGuid
    FROM apt29 a
    INNER JOIN (
      SELECT ProcessGuid
      FROM apt29
      WHERE Channel = "Microsoft-Windows-Sysmon/Operational"
          AND EventID = 1
          AND LOWER(Image) LIKE "%control.exe"
          AND LOWER(ParentImage) LIKE "%sdclt.exe"
    ) b
    ON a.ParentProcessGuid = b.ProcessGuid
    WHERE a.Channel = "Microsoft-Windows-Sysmon/Operational"
      AND a.EventID = 1
      AND a.IntegrityLevel = "High"
  ) c
  ON d.ParentProcessGuid= c.ProcessGuid
  WHERE d.Channel = "Microsoft-Windows-Sysmon/Operational"
    AND d.EventID = 1
    AND d.Image LIKE '%powershell.exe'
) e
ON f.ExecutionProcessID = e.ProcessId
WHERE f.Channel = "Microsoft-Windows-PowerShell/Operational"
  AND f.EventID = 4103
  AND LOWER(f.Payload) LIKE "%new-service%"
'''
)
df.show(100,truncate = False, vertical = True)

-RECORD 0------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 @timestamp | 2020-05-02T03:04:16.779Z                                                                                                                                                                                                                                                                                                                                                                   
 EventTime  | 2020-05-01 23:04:15                                                                                                                                                                                   

## DTR-0017-6106-5518
TTP de referência: TTP-0017-6046-4432

In [53]:
df = spark.sql(
'''
SELECT `@timestamp`,UtcTime,EventTime,Message
FROM apt29 f
INNER JOIN (
    SELECT d.ProcessGuid
    FROM apt29 d
    INNER JOIN (
      SELECT a.ProcessGuid, a.ParentProcessGuid
      FROM apt29 a
      INNER JOIN (
        SELECT ProcessGuid
        FROM apt29
        WHERE Channel = "Microsoft-Windows-Sysmon/Operational"
            AND EventID = 1
            AND LOWER(Image) LIKE "%control.exe"
            AND LOWER(ParentImage) LIKE "%sdclt.exe"
      ) b
      ON a.ParentProcessGuid = b.ProcessGuid
      WHERE a.Channel = "Microsoft-Windows-Sysmon/Operational"
        AND a.EventID = 1
        AND a.IntegrityLevel = "High"
    ) c
    ON d.ParentProcessGuid= c.ProcessGuid
    WHERE d.Channel = "Microsoft-Windows-Sysmon/Operational"
      AND d.EventID = 1
      AND d.Image LIKE '%powershell.exe'
) e
ON f.ProcessGuid = e.ProcessGuid
WHERE f.Channel = "Microsoft-Windows-Sysmon/Operational"
    AND f.EventID = 11
    AND f.TargetFilename RLIKE '.*\\\\\\\\ProgramData\\\\\\\\Microsoft\\\\\\\\Windows\\\\\\\\Start Menu\\\\\\\\Programs\\\\\\\\StartUp.*'
'''
)
df.show(100,truncate = False, vertical = True)

-RECORD 0---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 @timestamp | 2020-05-02T03:04:24.839Z                                                                                                                                                                                                                                                                                                                
 UtcTime    | 2020-05-02 03:04:23.681                                                                                                                                                                                                                                                                                     

## DTR-0017-6106-5995
TTP de referência: TTP-0017-6046-5816

In [54]:
df = spark.sql(
'''
SELECT `@timestamp`,UtcTime,EventTime,Message
FROM apt29 h
INNER JOIN (
    SELECT f.ProcessGuid
    FROM apt29 f
    INNER JOIN (
      SELECT d.ProcessGuid, d.ParentProcessGuid
      FROM apt29 d
      INNER JOIN (
        SELECT a.ProcessGuid, a.ParentProcessGuid
        FROM apt29 a
        INNER JOIN (
          SELECT ProcessGuid
          FROM apt29
          WHERE Channel = "Microsoft-Windows-Sysmon/Operational"
              AND EventID = 1
              AND LOWER(Image) LIKE "%control.exe"
              AND LOWER(ParentImage) LIKE "%sdclt.exe"
        ) b
        ON a.ParentProcessGuid = b.ProcessGuid
        WHERE a.Channel = "Microsoft-Windows-Sysmon/Operational"
          AND a.EventID = 1
          AND a.IntegrityLevel = "High"
      ) c
      ON d.ParentProcessGuid= c.ProcessGuid
      WHERE d.Channel = "Microsoft-Windows-Sysmon/Operational"
        AND d.EventID = 1
        AND d.Image LIKE '%powershell.exe'
    ) e
    ON f.ParentProcessGuid = e.ProcessGuid
    WHERE f.Channel = "Microsoft-Windows-Sysmon/Operational"
      AND f.EventID = 1
      AND LOWER(f.Image) LIKE '%accesschk%'
) g
ON h.ProcessGuid = g.ProcessGuid
WHERE h.Channel = "Microsoft-Windows-Sysmon/Operational"
    AND EventID = 7
    AND LOWER(ImageLoaded) LIKE '%accesschk%'
'''
)
df.show(100,truncate = False, vertical = True)

-RECORD 0---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 @timestamp | 2020-05-02T03:04:36.295Z                                                                                                                                                                                                                                                                                                                                                                   

## DTR-0017-6106-6816
TTP de referência: TTP-0017-6049-5413

In [55]:
df = spark.sql(
'''
SELECT `@timestamp`,UtcTime,EventTime,Message
FROM apt29 f
INNER JOIN (
    SELECT d.ProcessGuid, d.ParentProcessGuid
    FROM apt29 d
    INNER JOIN (
      SELECT a.ProcessGuid, a.ParentProcessGuid
      FROM apt29 a
      INNER JOIN (
        SELECT ProcessGuid
        FROM apt29
        WHERE Channel = "Microsoft-Windows-Sysmon/Operational"
            AND EventID = 1
            AND LOWER(Image) LIKE "%control.exe"
            AND LOWER(ParentImage) LIKE "%sdclt.exe"
      ) b
      ON a.ParentProcessGuid = b.ProcessGuid
      WHERE a.Channel = "Microsoft-Windows-Sysmon/Operational"
        AND a.EventID = 1
        AND a.IntegrityLevel = "High"
    ) c
    ON d.ParentProcessGuid= c.ProcessGuid
    WHERE d.Channel = "Microsoft-Windows-Sysmon/Operational"
      AND d.EventID = 1
      AND d.Image LIKE '%powershell.exe'
) e
ON f.ProcessGuid = e.ProcessGuid
WHERE f.Channel = "Microsoft-Windows-Sysmon/Operational"
    AND f.EventID = 7
    AND LOWER(f.ImageLoaded) LIKE "%system.drawing.ni.dll"
'''
)
df.show(100,truncate = False, vertical = True)

-RECORD 0---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 @timestamp | 2020-05-02T03:06:44.472Z                                                                                                                                                                               

## DTR-0017-6106-7870
TTP de referência: TTP-0017-6049-5413

In [56]:
df = spark.sql(
'''
SELECT `@timestamp`,EventTime,Payload
FROM apt29 f
INNER JOIN (
    SELECT d.ProcessId, d.ParentProcessId
    FROM apt29 d
    INNER JOIN (
      SELECT a.ProcessGuid, a.ParentProcessGuid
      FROM apt29 a
      INNER JOIN (
        SELECT ProcessGuid
        FROM apt29
        WHERE Channel = "Microsoft-Windows-Sysmon/Operational"
            AND EventID = 1
            AND LOWER(Image) LIKE "%control.exe"
            AND LOWER(ParentImage) LIKE "%sdclt.exe"
      ) b
      ON a.ParentProcessGuid = b.ProcessGuid
      WHERE a.Channel = "Microsoft-Windows-Sysmon/Operational"
        AND a.EventID = 1
        AND a.IntegrityLevel = "High"
    ) c
    ON d.ParentProcessGuid= c.ProcessGuid
    WHERE d.Channel = "Microsoft-Windows-Sysmon/Operational"
      AND d.EventID = 1
      AND d.Image LIKE '%powershell.exe'
) e
ON f.ExecutionProcessID = e.ProcessId
WHERE f.Channel = "Microsoft-Windows-PowerShell/Operational"
    AND f.EventID = 4103
    AND LOWER(f.Payload) LIKE "%copyfromscreen%"
'''
)
df.show(100,truncate = False, vertical = True)

-RECORD 0-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

## DTR-0017-6106-7534
TTP de referência: TTP-0017-6049-7247

In [57]:
df = spark.sql(
'''
SELECT `@timestamp`,EventTime,Payload
FROM apt29 f
INNER JOIN (
  SELECT d.ProcessId, d.ParentProcessId
  FROM apt29 d
  INNER JOIN (
    SELECT a.ProcessGuid, a.ParentProcessGuid
    FROM apt29 a
    INNER JOIN (
      SELECT ProcessGuid
      FROM apt29
      WHERE Channel = "Microsoft-Windows-Sysmon/Operational"
          AND EventID = 1
          AND LOWER(Image) LIKE "%control.exe"
          AND LOWER(ParentImage) LIKE "%sdclt.exe"
    ) b
    ON a.ParentProcessGuid = b.ProcessGuid
    WHERE a.Channel = "Microsoft-Windows-Sysmon/Operational"
      AND a.EventID = 1
      AND a.IntegrityLevel = "High"
  ) c
  ON d.ParentProcessGuid= c.ProcessGuid
  WHERE d.Channel = "Microsoft-Windows-Sysmon/Operational"
    AND d.EventID = 1
    AND d.Image LIKE '%powershell.exe'
) e
ON f.ExecutionProcessID = e.ProcessId
WHERE f.Channel = "Microsoft-Windows-PowerShell/Operational"
    AND f.EventID = 4103
    AND LOWER(f.Payload) LIKE "%get-clipboard%"
'''
)
df.show(100,truncate = False, vertical = True)

-RECORD 0---------------------------------------------------------
 @timestamp | 2020-05-02T03:06:43.474Z                            
 EventTime  | 2020-05-01 23:06:42                                 
 Payload    | CommandInvocation(Get-Clipboard): "Get-Clipboard"
 



## DTR-0017-6106-8933
TTP de referência: TTP-0017-6049-8462

In [58]:
df = spark.sql(
'''
SELECT `@timestamp`,EventTime,f.ScriptBlockText
FROM apt29 f
INNER JOIN (
  SELECT d.ProcessId, d.ParentProcessId
  FROM apt29 d
  INNER JOIN (
    SELECT a.ProcessGuid, a.ParentProcessGuid
    FROM apt29 a
    INNER JOIN (
      SELECT ProcessGuid
      FROM apt29
      WHERE Channel = "Microsoft-Windows-Sysmon/Operational"
          AND EventID = 1
          AND LOWER(Image) LIKE "%control.exe"
          AND LOWER(ParentImage) LIKE "%sdclt.exe"
    ) b
    ON a.ParentProcessGuid = b.ProcessGuid
    WHERE a.Channel = "Microsoft-Windows-Sysmon/Operational"
      AND a.EventID = 1
      AND a.IntegrityLevel = "High"
  ) c
  ON d.ParentProcessGuid= c.ProcessGuid
  WHERE d.Channel = "Microsoft-Windows-Sysmon/Operational"
    AND d.EventID = 1
    AND d.Image LIKE '%powershell.exe'
) e
ON f.ExecutionProcessID = e.ProcessId
WHERE f.Channel = "Microsoft-Windows-PowerShell/Operational"
    AND f.EventID = 4104
    AND LOWER(f.ScriptBlockText) LIKE "%compress-7zip%"
'''
)
df.show(100,truncate = False, vertical = True)

-RECORD 0-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

## DTR-0017-6106-9592
TTP de referência: TTP-0017-6049-8712

In [59]:
df = spark.sql(
'''
SELECT `@timestamp`,UtcTime,EventTime,Message
FROM apt29
WHERE Channel = "Microsoft-Windows-Sysmon/Operational"
    AND EventID = 1
    AND CommandLine RLIKE '.*rundll32.exe.*\\\\\\\\windows\\\\\\\\system32\\\\\\\\davclnt.dll.*DavSetCookie.*'
'''
)
df.show(100,truncate = False, vertical = True)

-RECORD 0-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

## DTR-0017-6107-0164
TTP de referência: TTP-0017-6049-9015

In [60]:
df = spark.sql(
'''
SELECT `@timestamp`,UtcTime,EventTime,Message
FROM apt29 f
INNER JOIN (
SELECT d.ProcessId, d.ParentProcessId
FROM apt29 d
INNER JOIN (
  SELECT a.ProcessGuid, a.ParentProcessGuid
  FROM apt29 a
  INNER JOIN (
    SELECT ProcessGuid
    FROM apt29
    WHERE Channel = "Microsoft-Windows-Sysmon/Operational"
        AND EventID = 1
        AND LOWER(Image) LIKE "%control.exe"
        AND LOWER(ParentImage) LIKE "%sdclt.exe"
  ) b
  ON a.ParentProcessGuid = b.ProcessGuid
  WHERE a.Channel = "Microsoft-Windows-Sysmon/Operational"
    AND a.EventID = 1
    AND a.IntegrityLevel = "High"
) c
ON d.ParentProcessGuid= c.ProcessGuid
WHERE d.Channel = "Microsoft-Windows-Sysmon/Operational"
  AND d.EventID = 1
  AND d.Image LIKE '%powershell.exe'
) e
ON f.ProcessId = e.ProcessId
WHERE f.Channel = "Microsoft-Windows-Sysmon/Operational"
  AND f.EventID = 3
  AND f.DestinationPort = 389
'''
)
df.show(100,truncate = False, vertical = True)

-RECORD 0-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 @timestamp | 2020-05-02T03:09:07.650Z                                                                                                                                                                                                                                                                                                                                                                                                                                                                             

## DTR-0017-6107-0843
TTP de referência: TTP-0017-6049-9628

In [61]:
df = spark.sql(
'''
SELECT `@timestamp`,UtcTime,EventTime,Message
FROM apt29 f
INNER JOIN (
SELECT d.ProcessId, d.ParentProcessId
FROM apt29 d
INNER JOIN (
  SELECT a.ProcessGuid, a.ParentProcessGuid
  FROM apt29 a
  INNER JOIN (
    SELECT ProcessGuid
    FROM apt29
    WHERE Channel = "Microsoft-Windows-Sysmon/Operational"
        AND EventID = 1
        AND LOWER(Image) LIKE "%control.exe"
        AND LOWER(ParentImage) LIKE "%sdclt.exe"
  ) b
  ON a.ParentProcessGuid = b.ProcessGuid
  WHERE a.Channel = "Microsoft-Windows-Sysmon/Operational"
    AND a.EventID = 1
    AND a.IntegrityLevel = "High"
) c
ON d.ParentProcessGuid= c.ProcessGuid
WHERE d.Channel = "Microsoft-Windows-Sysmon/Operational"
  AND d.EventID = 1
  AND d.Image LIKE '%powershell.exe'
) e
ON f.ProcessId = e.ProcessId
WHERE f.Channel = "Microsoft-Windows-Sysmon/Operational"
  AND f.EventID = 3
  AND f.DestinationPort = 5985
'''
)
df.show(100,truncate = False, vertical = True)

-RECORD 0------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 @timestamp | 2020-05-02T03:09:33.128Z                                                                                                                                                                                                                                                                                                                                                                                                                                                                            

## DTR-0017-6107-2091
TTP de referência: TTP-0017-6049-9772

In [62]:
df = spark.sql(
'''
SELECT `@timestamp`,EventTime,ScriptBlockText
FROM apt29 b
INNER JOIN (
    SELECT ProcessGuid, ProcessId
    FROM apt29
    WHERE Channel = "Microsoft-Windows-Sysmon/Operational"
    AND EventID = 1
    AND LOWER(Image) LIKE '%wsmprovhost.exe'
) a
ON b.ExecutionProcessID = a.ProcessId
WHERE b.Channel = "Microsoft-Windows-PowerShell/Operational"
  AND b.EventID = 4104
  AND LOWER(b.ScriptBlockText) LIKE "%get-process%"
'''
)
df.show(100,truncate = False, vertical = True)

-RECORD 0-------------------------------------------------------------------------------------------------------------------------------------------------------------------
 @timestamp      | 2020-05-02T03:09:29.929Z                                                                                                                                 
 EventTime       | 2020-05-01 23:09:28                                                                                                                                      
 ScriptBlockText |  Get-Process -IncludeUserName | Select-Object UserName,SessionId | Where-Object { $_.UserName -like "*\$env:USERNAME" } | Sort-Object SessionId -Unique  



## DTR-0017-6107-2726
TTP de referência: TTP-0017-6050-0048

In [64]:
df = spark.sql(
'''
SELECT `@timestamp`,EventTime, Hostname, ShareName, RelativeTargetName, SubjectUserName
FROM apt29
WHERE LOWER(Channel) = "security"
  AND EventID = 5145
  AND ShareName LIKE '%IPC%'
  AND RelativeTargetName LIKE '%PSEXESVC%'
'''
)
df.show(100,truncate = False, vertical = True)

-RECORD 0-------------------------------------------
 @timestamp         | 2020-05-02T03:11:41.250Z      
 EventTime          | 2020-05-01 23:11:40           
 Hostname           | NASHUA.dmevals.local          
 ShareName          | \\*\IPC$                      
 RelativeTargetName | PSEXESVC                      
 SubjectUserName    | pbeesly                       
-RECORD 1-------------------------------------------
 @timestamp         | 2020-05-02T03:11:41.253Z      
 EventTime          | 2020-05-01 23:11:40           
 Hostname           | NASHUA.dmevals.local          
 ShareName          | \\*\IPC$                      
 RelativeTargetName | PSEXESVC-SCRANTON-2668-stdin  
 SubjectUserName    | pbeesly                       
-RECORD 2-------------------------------------------
 @timestamp         | 2020-05-02T03:11:41.253Z      
 EventTime          | 2020-05-01 23:11:40           
 Hostname           | NASHUA.dmevals.local          
 ShareName          | \\*\IPC$                

## DTR-0017-6107-3395
TTP de referência: TTP-0017-6050-0265

In [65]:
df = spark.sql(
'''
SELECT `@timestamp`,UtcTime,EventTime,Message
FROM apt29 f
INNER JOIN (
    SELECT d.ProcessGuid
    FROM apt29 d
    INNER JOIN (
        SELECT b.ProcessGuid
        FROM apt29 b
        INNER JOIN (
          SELECT ProcessGuid
          FROM apt29
          WHERE Channel = "Microsoft-Windows-Sysmon/Operational"
              AND EventID = 1
              AND ParentImage LIKE '%services.exe'
        ) a
        ON b.ParentProcessGuid = a.ProcessGuid
        WHERE Channel = "Microsoft-Windows-Sysmon/Operational"
          AND Image LIKE '%python.exe'
    ) c
    ON d.ParentProcessGuid = c.ProcessGuid
    WHERE Channel = "Microsoft-Windows-Sysmon/Operational"
        AND EventID = 1
) e
ON f.ProcessGuid = e.ProcessGuid
WHERE Channel = "Microsoft-Windows-Sysmon/Operational"
    AND EventID = 11
'''
)
df.show(100,truncate = False, vertical = True)

-RECORD 0--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 @timestamp | 2020-05-02T03:15:40.849Z                                                                                                                                                                                                                                       
 UtcTime    | 2020-05-02 03:15:38.985                                                                                                                                                                                                                                        
 EventTime  | 2020-05-01 23:15:38                                                                                                                                                             

## DTR-0017-6107-4117
TTP de referência: TTP-0017-6053-3303

In [66]:
df = spark.sql(
'''
SELECT `@timestamp`,EventTime,ScriptBlockText
FROM apt29 h
INNER JOIN (
    SELECT f.ProcessId
    FROM apt29 f
    INNER JOIN (
      SELECT d.ProcessGuid
      FROM apt29 d
      INNER JOIN (
          SELECT b.ProcessGuid
          FROM apt29 b
          INNER JOIN (
            SELECT ProcessGuid
            FROM apt29
            WHERE Channel = "Microsoft-Windows-Sysmon/Operational"
                AND EventID = 1
                AND ParentImage LIKE '%services.exe'
          ) a
          ON b.ParentProcessGuid = a.ProcessGuid
          WHERE Channel = "Microsoft-Windows-Sysmon/Operational"
            AND Image LIKE '%python.exe'
      ) c
      ON d.ParentProcessGuid = c.ProcessGuid
      WHERE Channel = "Microsoft-Windows-Sysmon/Operational"
          AND EventID = 1
    ) e
    ON f.ParentProcessGuid = e.ProcessGuid
    WHERE Channel = "Microsoft-Windows-Sysmon/Operational"
      AND EventID = 1
      AND Image LIKE '%powershell.exe'
) g
ON h.ExecutionProcessID = g.ProcessId
WHERE h.Channel = "Microsoft-Windows-PowerShell/Operational"
    AND h.EventID = 4104
    AND LOWER(h.ScriptBlockText) LIKE "%childitem%"
'''
)
df.show(100,truncate = False, vertical = True)

-RECORD 0------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 @timestamp      | 2020-05-02T03:16:00.205Z                                                                                                                                                                                                                                                                                                                                                                                                                                      
 EventTime       | 2020-05-01 23:15:59                              

## DTR-0017-6107-4769
TTP de referência: TTP-0017-6053-5026

In [67]:
df = spark.sql(
'''
SELECT `@timestamp`,UtcTime,EventTime,Message
FROM apt29 h
INNER JOIN (
    SELECT f.ProcessGuid
    FROM apt29 f
    INNER JOIN (
      SELECT d.ProcessGuid
      FROM apt29 d
      INNER JOIN (
          SELECT b.ProcessGuid
          FROM apt29 b
          INNER JOIN (
            SELECT ProcessGuid
            FROM apt29
            WHERE Channel = "Microsoft-Windows-Sysmon/Operational"
                AND EventID = 1
                AND ParentImage LIKE '%services.exe'
          ) a
          ON b.ParentProcessGuid = a.ProcessGuid
          WHERE Channel = "Microsoft-Windows-Sysmon/Operational"
            AND Image LIKE '%python.exe'
      ) c
      ON d.ParentProcessGuid = c.ProcessGuid
      WHERE Channel = "Microsoft-Windows-Sysmon/Operational"
          AND EventID = 1
    ) e
    ON f.ParentProcessGuid = e.ProcessGuid
    WHERE Channel = "Microsoft-Windows-Sysmon/Operational"
      AND EventID = 1
      AND Image LIKE '%powershell.exe'
) g
ON h.ParentProcessGuid = g.ProcessGuid
WHERE Channel = "Microsoft-Windows-Sysmon/Operational"
    AND h.EventID = 1
    AND LOWER(h.CommandLine) LIKE "%rar.exe%"
'''
)
df.show(100,truncate = False, vertical = True)

-RECORD 0-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

## DTR-0017-6107-5368
TTP de referência: TTP-0017-6053-5447

In [68]:
df = spark.sql(
'''
SELECT `@timestamp`,UtcTime,EventTime,Message
FROM apt29 j
INNER JOIN (
    SELECT h.ProcessGuid
    FROM apt29 h
    INNER JOIN (
        SELECT f.ProcessGuid
        FROM apt29 f
        INNER JOIN (
          SELECT d.ProcessGuid
          FROM apt29 d
          INNER JOIN (
              SELECT b.ProcessGuid
              FROM apt29 b
              INNER JOIN (
                SELECT ProcessGuid
                FROM apt29
                WHERE Channel = "Microsoft-Windows-Sysmon/Operational"
                    AND EventID = 1
                    AND ParentImage LIKE '%services.exe'
              ) a
              ON b.ParentProcessGuid = a.ProcessGuid
              WHERE Channel = "Microsoft-Windows-Sysmon/Operational"
                AND Image LIKE '%python.exe'
          ) c
          ON d.ParentProcessGuid = c.ProcessGuid
          WHERE Channel = "Microsoft-Windows-Sysmon/Operational"
              AND EventID = 1
        ) e
        ON f.ParentProcessGuid = e.ProcessGuid
        WHERE Channel = "Microsoft-Windows-Sysmon/Operational"
          AND EventID = 1
          AND Image LIKE '%cmd.exe'
    ) g
    ON h.ParentProcessGuid = g.ProcessGuid
    WHERE Channel = "Microsoft-Windows-Sysmon/Operational"
        AND h.EventID = 1
) i
ON j.ProcessGuid = i.ProcessGuid
WHERE j.Channel = "Microsoft-Windows-Sysmon/Operational"
    AND j.EventID = 23
'''
)
df.show(100,truncate = False, vertical = True)

-RECORD 0-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 @timestamp | 2020-05-02T03:17:43.537Z                                                                                                                                                                                                                                                                                                                                                                                                                                 

## DTR-0017-6107-5740
TTP de referênica: TTP-0017-6053-5618

In [69]:
df = spark.sql(
'''
SELECT `@timestamp`,UtcTime,EventTime,Message
FROM apt29
WHERE Channel = "Microsoft-Windows-Sysmon/Operational"
  AND EventID = 1
  AND ParentImage LIKE '%services.exe'
  AND Image LIKE '%javamtsup.exe'
'''
)
df.show(100,truncate = False, vertical = True)

-RECORD 0-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 @timestamp | 2020-05-02T03:19:53.189Z                                                                                 